<img src="images/openmp.png"style="width:300px;">

# OpenMP: a short introduction

For this lab, you can watch the different videos proposed by Tim Mattson (Intel) on [YouTube](https://www.youtube.com/playlist?list=PLLX-Q6B8xqZ8n8bwjGdzBJ25X2utwnoEG)

Many tutorials are available online on [openmp.org](http://www.openmp.org/resources/tutorials-articles/)


## Moore's Law

To help you understand the Moore's Law, take a look at this [video](https://youtu.be/cMWGeJyrc9w).

## OpenMP API

<ol>
<li>__Directives__ to create parallel regions, synchronizations, or to define data status (private, share, ...)</li>
<li> __Library__ for specific functionalities (dynamic informations, runtime actions...)</li>
<li>__Environnement variables__ to refine program execution (threads numbers, schedule stratégies ...)</li>
</ol>
<img src="images/Api_OpenMP_En.png"style="width:800px;">


## OpenMP execution model
<ol>
<li>User include __directives__ to define __parallel regions__</li>
<li>During runtime, respect the so-called __fork-join__ model programming:</li>
<ul>
<li>Master thread creates a team of threads (*fork*)
<li>At the end of the parallel region only the master of the team continues, all others terminate (*join*)</li>
</ul></ol>
<img src="images/modele_execution_En.png" style="width:800px;">

## OpenMP Memory model

- all threads access the same __shared memory__; 
- every thread has its own __private memory__;
- __shared__ data is available to all threads;
- __private__ data is only available to the related thread; 
- Data transfer is transparent to the developer

<img src="images/mem_openmp_En.png" style="width:500px;">

## Main OpenMP directives
<ul>
<li>Building of parallel regions</li>
<ul>
    <li>``` parallel``` creates a parallel region based on fork-join model.</li>
</ul>
<li>Work sharing</li>
<ul>
  <li>```for ```: share iterations of a parallel loop;</li>
  <li>``` sections ```: define blocks that will be executed simultaneously;</li>
  <li>``` single ```: declare a block that will be executed by one thread only.</li>
</ul>
<li>Synchronisation</li>
<ul>
  <li>``` master ```: block that must be executed by the master thread;</li>
  <li>``` critical ``` : block that can be executed only by one thread at a time;</li>
  <li>``` atomic ``` : atomic write memory instruction;   
  <li>``` barrier ```: wait until all threads reach that point.</li>
</ul><li>Tasks management</li>
<ul>
  <li>``` task ```: descendant task declaration;</li>
  <li>``` taskwait ```: wait the end of the descendant task.</li>
</ul></ul>

# OpenMP Directives

__C/C++ directives format__

```C
#pragma omp directive [clause [clause] ...]
```
## Comprising four parts :
1. OpenMP tag: ```#pragma omp```
2. Valid directive name
3. Optional list clauses (additionals informations)
4. A newline

## General rules :
<ul>
<li>OpenMP C/C++ directives are case-sensitive;</li>
<li>An OpenMP directive apply to the next code block;</li>
<li>Long directives can be written on multi-lines by adding a backslash « \ » at the end of the line.</li>
</ul>

## Parallel directive

```C
#pragma omp parallel [clause [clause] ... ]
{
    // Parallel region
}
```
**Function :**
- When a thread reaches the ``` parallel ``` directive, it creates a team of threads and acts as the master thread; threads of the team execute all blocks;
- Number of threads depends in the order on, the ``` if ``` evaluation claude, the ``` num_threads ``` clause, the  ``` omp_set_num_threads()``` primitive,  the ``` OMP_NUM_THREADS``` environnement variable, the default value;
- There is an implicit barrier at the end of the parallel region;
- By default, variables status is __shared__ in parallel regions;
- However, if the region contains functions calls, their locals and automatics variables are private;
- It is not allowed to jump from or to a parallel region (goto)
- Possibles clauses : ``` if, num_threads, private, shared, default, firstprivate, reduction, copyin ```

### Parallel regions in detail

```C
#pragma omp parallel
    {
        printf ("The parallel region is executed by the thread %d\n",omp_get_thread_num());
    }  // end of parallel region
```

The structure ``` omp parallel``` supports different additional clauses:

- ```if (expression) ``` : 
- ``` num_threads (integer) ```:
- ``` private (list) ```:
- ```firstprivate (list) ```:
- ```lastprivate (list) ```:
- ```shared (list) ```:
- ```default (none | shared)```:
- ```copyin (list) ```:
- ```reduction (operator:list)```:

### The if clause 

** if clause format in C/C++ **
```C
if (/* Scalar expression */)
```
When this clause is present, the thread team is only created if 'scalar expression' is non equal to zero, else the region is sequentialy executed by the master thread

** If clause exemple **
```C
#include <stdio .h>
#define PARALLEL 1 // 0 for sequential , != 0 for parallel

int main() {
    #pragma omp parallel if (PARALLEL)
    printf("Hello, openMP\n");
    return 0; 
}
```

### The num_threads clause

** num_threads clause format in C/C++ **
```C
num_threads (/* Integer expression */)
```

- Specifies the number of threads that will execute the next parallel region;
- Expression must be evaluated as an positive integer value.


### Exercice 1:

```C
#include <stdio .h>

int main() 
{
    #pragma omp parallel
    printf("Hello world\n" ); 
    return 0;
}
```

- Compile this program (file hello.c in exercice1 folder) with and without ```-fopenmp ``` option
- Run program in both two cases
- What is the default theads number ? Is that raisonable?
- Change the number of threads used by your program.

In [1]:
editor()

In [2]:
%%bash
cd exercice1
gcc hello.c -o hello
echo "Done"

Done


In [3]:
%%execute
cd exercice1
export OMP_NUM_THREADS=8
./hello

Submitted batch job 994151
Waiting for resources .........o
End batch job 994151 Status:  COMPLETED 

Slurm command was : sbatch -n 1 --output=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.out --error=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.err
Hello world


# Work sharing

- Includes in all parallel regions
- Has to be met by all or none of the threads
- Shares the work between the deferent threads
- Implies a barrier at the end of the construction (except if the nowait clause is present).

<img src="images/partage_travail_En.png" style="width:1000px;">

There are three basic methods of sharing : 

- ```C 
#pragma omp for ```
- ```C
#pragma omp sections ```
- ```C
#pragma omp single ```

Behavior of each structure can be specified by clauses that will be defined later in this course

## For directive

```C
#pragma omp for [clause [clause] ... ]
for (init; condition; increment) 
{
    // loop code 
}
```

<ul>
<li> Indicates that loop iterations following the directive must be executed in parallel by a team of threads; </li>
<li> Iteration variables are private by default;</li>
<li> Loops must have a simple iterative form:</li>
<ul>
<li>The condition has to be the same for all threads</li>
<li>Unbounded loop or while loop are not allowed</li>
</ul>

<li>**Reminder** : the developer is responsible for the code semantics</li>
<li>Clauses possibles : ``` schedule, ordered, private, firstprivate, lastprivate, reduction, collapse, nowait ```</li>
</ul>

### Exercice 2

- Write a C code that sums each component of an array with a scalar value and write the result in a second array;
- If needed, an initial sequential C program is providen in exercice2 folder;
- Parallelize that code with **OpenMP**.

In [4]:
editor()

In [5]:
%%bash
cd exercice2
gcc exercice2.c -o exercice2 -fopenmp -std=c99 
echo "Done"

Done


In [6]:
%%execute
cd exercice2
./exercice2

Submitted batch job 994152
Waiting for resources ........
End batch job 994152 Status:  COMPLETED 

Slurm command was : sbatch -n 1 --output=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.out --error=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.err
Sequential execution
Initial array:
[0 1 2 3 4 5 6 7 8 9]
5 has been added to each element.
Final array:
[5 6 7 8 9 10 11 12 13 14]


### Schedule clause 

** Schedule clause format in C/C++ **

<ul>
<li>The schedule clause is supported on the loop construct only. It is used to control the manner in which loop iterations are distributed over the threads;</li>  

<ul>
<li>** static :  ** Iterations are divided into chunks of size __chunk_size__. The chunks are assigned to the threads statically in a round-robin manner, in the order of the threads numbers. When no __chunk_size__ is specified, the iteration space is devided into chunks that are approximately equal in size. Each thread is assigned at most one chunk;</li>
<li>** dynamic : ** The iterations are assigned to threads as the threads request them. The thread executes the chunk of iterations, then request another chunk until there are no more chunk to work on. When no chunk_size is specified, it default is to 1;</li>
<li>** guided : ** The difference with dynamic is that with guided schedule, the size of the chunk of iteration decrease over time;</li>

<li>** runtime : **If this schedule is selected, the decision regarding scheduling kind is made at runtime. The schedule and (optional) chunk size are set through the OMP_SCHEDULE environnement variable;</li>

</ul>

### Exercice 3
```C
#include <stdio.h> 
#include <omp.h> 

#define SIZE 100 
#define CHUNK 10

int main() { 
    int tid;
    double a[SIZE],b[SIZE], c[SIZE];
    
    for (size_t i =0; i < SIZE; i++)
        a[i] = b[i] = i;
        
    #pragma omp parallel private(tid) 
    {
        tid = omp_get_thread_num () ; 
        if (tid == 0)
            printf("Nb threads = %d\n", omp_get_num_threads());
        printf("Thread %d: starting...\n", tid);
    #pragma omp barrier

    #pragma omp for schedule(dynamic, CHUNK) 
        for (size_t i = 0; i < SIZE; i++) {
            c[i] = a[i] + b[i];
            printf("Thread %d: c[%2zu] = %g\n", tid, i, c[i]); 
        }
    }
    return 0;
}
```

- Examine the program : which instructions are executed by all threads? By only one thread ?
- Run the program several times. What do you think about the instructions execution order ?
- Redirect the output into the __sort__ utility. Run the code and observe the iterations repartition.
- Repeat several times. Is the work repartition stable?
- Change the schedule policy and use static. Run the code several times. Is the work repartition stable ? 
- Comment the impact of the schedule policy on excution performances.

In [7]:
editor()

In [8]:
%%bash
cd solutions
gcc exercice3Omp.c -o exercice3Omp -fopenmp -std=c99 
echo "Done"

Done


In [9]:
%%execute
cd solutions
./exercice3Omp 8 

Submitted batch job 994153
Waiting for resources .........o
End batch job 994153 Status:  COMPLETED 

Slurm command was : sbatch -n 1 --output=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.out --error=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.err
Nb threads = 8
Thread 0: starting...
Thread 4: starting...
Thread 5: starting...
Thread 6: starting...
Thread 7: starting...
Thread 3: starting...
Thread 2: starting...
Thread 1: starting...
Thread 0: c[ 0] = 0
Thread 0: c[ 1] = 2
Thread 0: c[ 2] = 4
Thread 0: c[ 3] = 6
Thread 0: c[ 4] = 8
Thread 0: c[ 5] = 10
Thread 0: c[ 6] = 12
Thread 0: c[ 7] = 14
Thread 0: c[ 8] = 16
Thread 0: c[ 9] = 18
Thread 0: c[10] = 20
Thread 0: c[11] = 22
Thread 0: c[12] = 24
Thread 0: c[13] = 26
Thread 0: c[14] = 28
Thread 0: c[15] = 30
Thread 0: c[16] = 32
Thread 0: c[17] = 34
Thread 0: c[18] = 36
Thread 0: c[19] = 38
Thread 0: c[20] = 40
Thread 0: c[21] = 42
Thread 0: c[22] = 44
Thread 0: c[23] = 46
Thread 0: c[24] =

### Collapse clause

** Collapse clause format in C/C++ **
```C
collapse(/∗ Integer expression strictly positive ∗/)
```
Collapse clause might be added to a loop nest to enable parallelization of multiple loop levels.
- Specify the number of ```for ```  loop levels to be collapsed (default is 1);
- If the integer expression is 1, all loops are collapsed in order to create a single iterations space that will be distributed between all threads;
- The iteration order in the collapsed loops is the same as in the originals loops.


### Exercice 4:
- Write a sequential program that will compute the product of two matrices A and B. Put the code in the directory exercice4;
- Parallelize the previous code without using the ```collapse clause ``` directive;
- Parallelize the code using ```collapse clause ``` this time and with and without the ```collapse ``` clause ;
- Compare in each cases the execution time. How does treatement time change according to the number of threads used?

In [10]:
editor()

In [11]:
%%bash
cd exercice4
gcc -fopenmp exercice4.c -o exercice4
./exercice4

bash: line 2: cd: exercice4: No such file or directory
gcc: exercice4.c: No such file or directory
gcc: no input files
bash: line 4: ./exercice4: No such file or directory


### Nowait clause

- Suppress the barrier at the end of the work-sharing construct;
- When threads reach the end of the construct, they immediately proceed to perform other work;

For example,  when a thread is finished with the work associated with a parallelized ```for``` loop, it continues and no longer waits for other threads to finish as well.
```C
#include <stdio.h> 
#include <stdlib.h>
#include <omp.h>

#define SIZE 100

int main() {
    double a[SIZE], b[SIZE], c[SIZE], d[SIZE];
    for (size_t i = 0; i < SIZE; i++) a[i] = b[i] = i;
    #pragma omp parallel
    {
        #pragma omp for schedule(static) nowait 
            for (size_t i = 0; i < SIZE; i++)
                c[i] = a[i] + b[i];
        #pragma omp for schedule(static) 
            for (size_t i = 0; i < SIZE; i++)
                d[i] = a[i] + c[i]; 
     }
     for (size_t i = 0; i < SIZE; i++) 
         printf("%g ", d[i]);
     printf("\n");
   
     return EXIT_SUCCESS; 
}
```

### Exercice 5:
- Execute the program several times. Is the result coherent ?
- Examine the program : which iterations will be executed by which threads ?
- Is the use of the nowaite clause a good strategy ?
- Change the schedule policy in __guided__ for the second loop.
- Execute the program several times. Is the result correct? If you cannot find a problem, look better ;-) !

In [12]:
editor()

In [13]:
%%bash
cd solutions
gcc exercice5.c -o exercice5 -fopenmp -std=c99 
echo "Done"

Done


In [14]:
%%execute
cd solutions
./exercice5

Submitted batch job 994154
Waiting for resources .........oo
End batch job 994154 Status:  COMPLETED 

Slurm command was : sbatch -n 1 --output=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.out --error=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.err
0 3 6 9 12 15 18 21 24 27 30 33 36 39 42 45 48 51 54 57 60 63 66 69 72 75 78 81 84 87 90 93 96 99 102 105 108 111 114 117 120 123 126 129 132 135 138 141 144 147 150 153 156 159 162 165 168 171 174 177 180 183 186 189 192 195 198 201 204 207 210 213 216 219 222 225 228 231 234 237 240 243 246 249 252 255 258 261 264 267 270 273 276 279 282 285 288 291 294 297 


## Section directive
** Section directive format in C/C++ **
```C
#pragma omp sections [ clause [ clause ] . . . ] 
{
    #pragma omp section
        // Bloc 1
        ...
    #pragma omp section
        // Bloc N
}
```
- Specifies different code regions, each of which will be executed by one of the threads;
- Each section is executed one time
- Possible clauses : private, firstprivate, lastprivate, reduction, nowait.

**Exemple :**
```C
#pragma omp parallel
{
    #pragma omp sections
    {
        #pragma omp section
            (void) functionA();
            
        #pragma omp section 
            (void) functionB();
     }  /* end of section */
} /* end of parallel region */
```

## Single directive
** Single directive format in C/C++ **
```C
    #pragma omp single [ clause [ clause ] . . . ] {
        // Bloc
}
```
- Specifies that this block should be executed by one thread only;
- It does not state which thread should executes the code block;
- Usefull for non thread-safe parts of code (input/output for exemple)
- Possible clauses : private, firstprivate, copyprivate, nowait.

**Exemple :**
```C
#pragma omp parallel shared (a,b) private (i)

    pragma omp single 
    {
        a = 10;
        printf ("Bloc single is executed by the thread %d\n",
                omp_get_thread_num());
    }
    
    /* barrier */
    #pragma omp for
    {
        for (i=0;i<n;i++) 
            b[i]=a;
    }
    
}
```

### Exercice 6:
- with the next editor, create an empty folder exercice6 and add anempty file exercice6.c;
- copy/paste the example;
- compile and test it.


In [15]:
editor()

In [16]:
%%bash
cd exercice6
gcc

bash: line 2: cd: exercice6: No such file or directory
gcc: no input files


# Shortcurt for parallel for/sections
** Parallel for directive format**
```C
    #pragma omp parallel for [clause [clause] ...] 
    ...
```

** Parallel section directive format**
```C
    #pragma omp parallel sections [ clause [ clause ] 
    {
        #pragma omp section
            // Bloc 1
            ...
        #pragma omp section
            // Bloc N
    }
```

- Creates a parallel region with only one directive;
- Possible clauses : all combination of clauses except nowait.

# Approximation of Pi using the Monte Carlo method

- in the folder solutions, you can edit the pi_mc.c file;
- this program will estimate the value of Pi thanks to the Monte Carlo method;
- if you're not familiar with this method, you can go [there](https://curiosity-driven.org/pi-approximation) for example;
- One can decide to write is own code without editing to providen one...

In [17]:
editor()

In [18]:
%%bash
cd solutions
gcc pi_mc.c -o pi_mc -fopenmp -std=c99 
echo "Done"

Done


In [19]:
%%execute
cd solutions
./pi_mc 100000000 8

Submitted batch job 994156
Waiting for resources ........
Running .........ooo
End batch job 994156 Status:  COMPLETED 

Slurm command was : sbatch -n 1 --output=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.out --error=/home/lab00064/UTP_2017/IntroductionOpenMP/python-execute-slurm/%J.err
#threads = 8
Estimated value for Pi: 3.142335, average execution time: 3.124511
